<a href="https://colab.research.google.com/github/SridharSeshadri56/Decision_Models/blob/main/pyomoMinCostFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Minimum Cost Flow Problem: Minimize the cost of satsifying demand at a set of nodes given a supply network
# Define a set of nodes, their demand (>0 supply, <0 demand, =0 transhipment node)
# Define the set of arcs in the netwrk, capacities of arcs and cost per unit of transportation
# We model the constraints as net flow at each node <= demand

In [ ]:
pip install pyomo

In [ ]:
!apt-get install -y -qq glpk-utils  #Installs the optimization engine called glpk.

In [ ]:
from pyomo.environ import *
nodes = ['F1', 'F2', 'DC', 'W1', 'W2']     #The nodes in the network

#The arcs in the network which will serve to define decision variables
arcs = { ('F1','W1'), ('F1', 'DC'),('F2','DC'), \
        ('F2','W2'), \
        ('DC','W1'), \
        ('DC','W2') }

                         #Max flow on arcs  (I set capacity to large number (1000) when there is no limit on flow)
arc_capacity =  { ('F1','W1'): 1000, ('F1', 'DC'):50,('F2','DC'):50, \
        ('F2','W2'):1000, \
        ('DC','W1'):50, \
        ('DC','W2'):50 }

 # Demand > 0 means supply node, < 0 means demand, = 0 means transhipment node
demand = {'F1': 80, 'F2': 70, 'DC': 0, 'W1': -60, 'W2': -90}

 #The cost per unit transported on each arc
cost_per_unit = { ('F1','W1'): 700, ('F1', 'DC'): 300, \
                 ('F2','DC'): 500, \
                 ('F2','W2'): 1000, \
                 ('DC','W1'): 200, \
                 ('DC','W2'):400 }

model = ConcreteModel(name = "(Model2)")                #Object to define the model (any name on left hand side ok)

model.x = Var( arcs, within= NonNegativeReals )         #The decision variables are flows in the arcs

model.value = Objective(
expr = sum( cost_per_unit[i]*model.x[i] for i in arcs), sense = minimize )  # Minimize cost of satisfying demand

def one_per_node(m,c):                          # For any node outflow - inflow <= demand
    return sum(m.x[i,j] for (i,j) in arcs if i == c) - sum(m.x[i,j] for (i,j) in arcs if j == c) <= demand [c]

def one_per_arc(m,c1, c2):
    return m.x[c1, c2] <= arc_capacity[c1, c2]

model.one_per_node = Constraint(nodes, rule = one_per_node)   #Net flow should be less than or equal to demand
model.one_per_arc  = Constraint(arcs, rule = one_per_arc)     #Max flow on arc can not exceed arc capacity

opt = SolverFactory('glpk')

model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
results = opt.solve(model, tee= True)

In [ ]:
model.pprint()